In [263]:
import datetime
import logging
import os
import pathlib
import pprint
import re
import shutil
import subprocess
import sys
import typing as t
import xml.etree.ElementTree as ET

import git
import pandas as pd

In [2]:
_LOG = logging.getLogger('profile_flash')

In [3]:
try:
    _HERE = pathlib.Path(__file__).parent.resolve()
except NameError:
    _HERE = pathlib.Path(os.getcwd()).resolve()
_HERE

PosixPath('/nfs2/mbysiek/Projects/docker-transpyle-flash')

In [4]:
_RESULTS_ROOT = pathlib.Path(_HERE, 'results')

In [5]:
_CACHED = {}

In [6]:
_CACHED

{}

check if the envvars are properly set

In [7]:
os.environ['CPATH']

'/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/papi-5.6.0-5p5d3qi2imn6m3szyl47gj73dpixhxpf/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hpctoolkit-master-wxx6jn7xuoo7lzqhg7kwxx4xl2nrlooc/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/amrex-18.08.1-crdvjfdihxbjlam4swf2jzopgjv3dopv/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hypre-2.14.0-vea6lmriirgjzt4o6j4zxw2xbunynfs6/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/openblas-0.3.2-femnsy3tmxlxehbgrk4yd4gmig2zjsle/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hdf5-1.8.20-p357p5eclyvy4vpqaownurdkn3mxv56v/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/zlib-1.2.11-z7cqiqbuz4k4x4ideyvz3syxata5g5gd/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/numactl-2.0.11-nawuf6bcuc

In [8]:
os.environ['LD_LIBRARY_PATH']

'/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/papi-5.6.0-5p5d3qi2imn6m3szyl47gj73dpixhxpf/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hpctoolkit-master-wxx6jn7xuoo7lzqhg7kwxx4xl2nrlooc/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/amrex-18.08.1-crdvjfdihxbjlam4swf2jzopgjv3dopv/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hypre-2.14.0-vea6lmriirgjzt4o6j4zxw2xbunynfs6/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/openblas-0.3.2-femnsy3tmxlxehbgrk4yd4gmig2zjsle/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hdf5-1.8.20-p357p5eclyvy4vpqaownurdkn3mxv56v/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/zlib-1.2.11-z7cqiqbuz4k4x4ideyvz3syxata5g5gd/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/numactl-2.0.11-nawuf6bcuc5wuk7t5oncoltovb532qba/lib:/

In [9]:
shutil.which('hpcrun')

'/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hpctoolkit-master-wxx6jn7xuoo7lzqhg7kwxx4xl2nrlooc/bin/hpcrun'

runners implementation

In [10]:
def _run_and_check(cmd: str, wd: pathlib.Path, *,
                   test_name: str, phase_name: str):
    _LOG.warning('%s.%s: running "%s" with wd="%s"', test_name, phase_name, cmd, wd)
    # _dir = os.getcwd()
    # os.chdir(str(wd))
    cmd_result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                                shell=True, cwd=str(wd))
    # os.chdir(_dir)
    log_dir = logs_path(test_name=test_name)
    log_dir.mkdir(parents=True, exist_ok=True)
    cmd_stdout = cmd_result.stdout.decode()
    with open(str(pathlib.Path(log_dir, '{}_stdout.log'.format(phase_name))),
              'w') as cmd_stdout_file:
        cmd_stdout_file.write(cmd_stdout)
    cmd_stderr = cmd_result.stderr.decode()
    with open(str(pathlib.Path(log_dir, '{}_stderr.log'.format(phase_name))),
              'w') as cmd_stderr_file:
        cmd_stderr_file.write(cmd_stderr)
    cmd_msg = None
    if cmd_result.returncode != 0:
        cmd_msg = '"{}" failed, returncode={}, logs were written to "{}"' \
            ' and last 50 lines of stderr follow:\n{}'.format(
                cmd, cmd_result.returncode, log_dir,
                ''.join(cmd_stderr.splitlines(keepends=True)[-50:]))
    assert cmd_result.returncode == 0, cmd_msg

In [11]:
def setup_flash(experiment, setup_dir, *,
                test_name: str, phase_name: str = 'setup'):
    _run_and_check('./setup -site spack ' + experiment, setup_dir,
                   test_name=test_name, phase_name=phase_name)


def make_flash(build_dir, *,
                test_name: str, phase_name: str = 'make'):
    _run_and_check('make', build_dir,
                   test_name=test_name, phase_name=phase_name)


def clean_flash(build_dir, *,
                test_name: str, phase_name: str = 'clean'):
    _run_and_check('make clean', build_dir,
                   test_name=test_name, phase_name=phase_name)

In [12]:
def hpctoolkit_profile(executable: pathlib.Path, results_path: pathlib.Path, sample_size: int, mpi_proc: int = 0, *,
                       test_name: str, phase_name: str = 'profile'):
    hpcrun = 'hpcrun' # shutil.which('hpcrun')
    results_path.parent.mkdir(exist_ok=True)
    hpcrun_command = '{} -o "{}" {}'.format(hpcrun, results_path, executable)
    if with_mpi > 0:
        hpcrun_command = 'mpirun -np {} {}'.format(mpi_proc, hpcrun_command)
    _LOG.warning('%s.%s: running the experiment %i times...', test_name, phase_name, sample_size)
    for i in range(sample_size):
        _run_and_check(hpcrun_command, executable.parent,
                       test_name=test_name, phase_name=phase_name)
        # subprocess.run(hpcrun_command, shell=True)

In [13]:
def hpctoolkit_summarize(executable: pathlib.Path, results_path: pathlib.Path, source_path: pathlib.Path, *,
                         test_name: str, phase_name: str = 'summarize'):
    hpcstruct = 'hpcstruct'  # shutil.which('hpcstruct')
    hpcprof = 'hpcprof' # shutil.which('hpcprof')
    #source_path.joinpath('*')
    struct_path = results_path.joinpath(executable.name + '.hpcstruct')
    hpcstruct_command = '{} --verbose -I "{}" -o {} {}'.format(
        hpcstruct, './*', struct_path, executable)
    _run_and_check(hpcstruct_command, source_path,
                   test_name=test_name, phase_name='{}.hpcstruct'.format(phase_name))
    # hpcprof -I "${source}" "${results_path}" \
    #   -S "${results_path}/flash4.hpcstruct" \
    #   -M stats \
    #   -o "${results_path}_db"
    # source_path.joinpath('+')
    hpcprof_command = '{} --verbose -I "{}" {} -S {} -M stats -o {}'.format(
        hpcprof, './+', results_path, struct_path, profile_db_path(test_name=test_name))
    _run_and_check(hpcprof_command, source_path,
                   test_name=test_name, phase_name='{}.hpcprof'.format(phase_name))

In [14]:
def profile_flash(app_name: str, executable: pathlib.Path, source_path: pathlib.Path, sample_size: int, *,
                  test_name: str):
    results_path = profile_path(test_name=test_name)
    hpctoolkit_profile(executable, results_path, sample_size, test_name=test_name)
    hpctoolkit_summarize(executable, results_path, source_path, test_name=test_name)

In [15]:
def profile_experiment(app_name: str, experiment: str, branch: str, sample_size: int, *,
                       test_name: str):
    app_dir = pathlib.Path(_HERE, app_name)
    setup_dir = {
        'flash-subset': app_dir.joinpath('FLASH4.4')
        }.get(app_name, app_dir)
    repo = git.Repo(str(app_dir))
    assert not repo.is_dirty(untracked_files=True), repo
    if str(repo.active_branch) != branch:
        _LOG.warning('%s: checking out %s', test_name, branch)
        repo.git.checkout(branch)
    build_dir = setup_dir.joinpath('object')
    executable = build_dir.joinpath('flash4')
    setup_flash(experiment, setup_dir,
                test_name=test_name)
    make_flash(build_dir,
               test_name=test_name)
    profile_flash(app_name, executable, app_dir, sample_size,
                  test_name=test_name)
    clean_flash(build_dir,
                test_name=test_name)

In [22]:
def date_str(date=None) -> str:
    if date is None:
        date = _NOW
    return date.strftime('%Y%m%d-%H%M%S')


def logs_path(date=None, *, test_name) -> pathlib.Path:
    return _RESULTS_ROOT.joinpath('{}_{}'.format(date_str(date), test_name))


def profile_path(date=None, *, test_name) -> pathlib.Path:
    return _RESULTS_ROOT.joinpath('profile_{}_{}'.format(date_str(date), test_name))


def profile_db_path(date=None, *, test_name) -> pathlib.Path:
    profile_dir = profile_path(date, test_name=test_name)
    return profile_dir.with_name(profile_dir.name + '_db')

run the experiments

In [60]:
_NOW = datetime.datetime.now()

In [82]:
_NOW

datetime.datetime(2018, 9, 5, 12, 5, 19, 934255)

In [54]:
profile_experiment('flash-subset', 'Sedov -auto -2d +Mode1', 'transpyle_experiments_outlined', 50,
                   test_name='subset_sedov_outlined_a')

subset_sedov_outlined_a.profile: running the experiment 1 times...
subset_sedov_outlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-114027_subset_sedov_outlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_sedov_outlined_a.summarize.hpcstruct: running "hpcstruct --verbose -I "./*" -o /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-114027_subset_sedov_outlined_a/flash4.hpcstruct /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset"
subset_sedov_outlined_a.summarize.hpcprof: running "hpcprof --verbose -I "./+" /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-114027_subset_sedov_outlined_a -S /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_201809

In [ ]:
clean_flash(pathlib.Path(_HERE, 'flash-subset', 'FLASH4.4', 'object'), 'subset_sedov_outlined_a')

In [54]:
profile_experiment('flash-subset', 'Sod -auto -2d +Mode1', 'transpyle_experiments_outlined', 50,
                   test_name='subset_sedov_outlined_a')

subset_sedov_outlined_a.profile: running the experiment 1 times...
subset_sedov_outlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-114027_subset_sedov_outlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_sedov_outlined_a.summarize.hpcstruct: running "hpcstruct --verbose -I "./*" -o /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-114027_subset_sedov_outlined_a/flash4.hpcstruct /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset"
subset_sedov_outlined_a.summarize.hpcprof: running "hpcprof --verbose -I "./+" /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-114027_subset_sedov_outlined_a -S /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_201809

In [ ]:
profile_experiment('flash-subset', 'Sedov -auto -2d +Mode1', 'transpyle_experiments_autoinlined', 50,
                   test_name='subset_sedov_autoinlined_a')

In [ ]:
profile_experiment('flash-subset', 'Sod -auto -2d +Mode1', 'transpyle_experiments_autoinlined', 50,
                   test_name='subset_sedov_autoinlined_a')

In [61]:
for branch, nickname in {
        'transpyle_experiments': 'base', 'transpyle_experiments_outlined': 'outlined',
        'transpyle_experiments_autoinlined': 'autoinlined'}.items():
    for problem in ['Sod', 'Sedov']:
        test_name = 'subset_{}_{}_b'.format(problem, nickname)
        profile_experiment('flash-subset', '{} -auto -2d +Mode1'.format(problem), branch, 50,
                           test_name=test_name)

subset_Sod_base_a.setup: running "./setup -site spack Sod -auto -2d +Mode1" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4"
subset_Sod_base_a.make: running "make" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_base_a.profile: running the experiment 50 times...
subset_Sod_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_base_a.profile: running "

subset_Sod_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Pro

subset_Sod_base_a.clean: running "make clean" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_base_a.setup: running "./setup -site spack Sedov -auto -2d +Mode1" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4"
subset_Sedov_base_a.make: running "make" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_base_a.profile: running the experiment 50 times...
subset_Sedov_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subse

subset_Sedov_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_base_a.profile: running "hpcrun -o "/nf

subset_Sedov_base_a.summarize.hpcprof: running "hpcprof --verbose -I "./+" /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a -S /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a/flash4.hpcstruct -M stats -o /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a_db" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset"
subset_Sedov_base_a.clean: running "make clean" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_outlined_a: checking out transpyle_experiments_outlined
subset_Sod_outlined_a.setup: running "./setup -site spack Sod -auto -2d +Mode1" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4"
subset_Sod_outlined_a.make: running "make" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_outlined_a.profile: running t

...

subset_Sedov_outlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_outlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_outlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_outlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_outlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_outlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_outlined_a.summ

subset_Sod_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_autoinlined

subset_Sod_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_autoinlined

subset_Sedov_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sed

subset_Sedov_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sed

In [308]:
class HPCtoolkitDB(pd.DataFrame):

    """Parser of HPCtoolkit XML database files."""

    _metadata = ['_metrics_by_id', '_metrics_formulas', '_procedures_by_id']

    def __init__(self, *args, path: pathlib.Path = None, **kwargs):
        if path is None:
            super().__init__(*args, **kwargs)
            return

        profile_data = self._read_xml(path).find('./SecCallPathProfile')
        _LOG.warning('%s', profile_data.attrib['n'])

        metrics = profile_data.find('./SecHeader/MetricTable')
        _LOG.debug('%s', [(_, _.attrib) for _ in metrics])
        self._metrics_by_id = {int(_.attrib['i']): _.attrib['n'] for _ in metrics}
        _LOG.info('%s', pprint.pformat(self._metrics_by_id))

        self._metrics_formulas = self._derive_metrics_formulas(metrics)
        _LOG.warning('%s', pprint.pformat(self._metrics_formulas))

        procedures = profile_data.find('./SecHeader/ProcedureTable')
        _LOG.debug('%s', [(_, _.attrib) for _ in procedures])
        self._procedures_by_id = {int(_.attrib['i']): _.attrib['n'] for _ in procedures}
        _LOG.info('%s', pprint.pformat(self._procedures_by_id))

        measurements = profile_data.find('./SecCallPathProfileData')
        _LOG.debug('%s', [(_, _.attrib) for _ in measurements])

        super().__init__(data=None, index=None, columns=[metric for _, metric in sorted(self._metrics_by_id.items())])

        #setattr(self, '_metrics_by_id', _metrics_by_id)

        self._add_measurements(measurements, 'global', depth=0)

    def _read_xml(self, path: pathlib.Path) -> ET.ElementTree:
        with path.open() as trace_file:
            xml_data = ET.parse(trace_file)
        return xml_data

    def _derive_metrics_formulas(self, metrics) -> dict:
        metrics_formulas = {}
        for metric in metrics:
            formulas = metric.findall('./MetricFormula')
            for formula in formulas:
                if formula.attrib['t'] != 'finalize':
                    continue
                raw_formula = formula.attrib['frm']
                _LOG.debug('%s', raw_formula)
                metrics_formulas[metric.attrib['n']] = \
                    re.sub('\$[0-9]+', lambda _: 'data[self._metrics_by_id[{}]]'.format(_.group()[1:]), raw_formula)
        return metrics_formulas
    
    def _evaluate_measurements_data(self, data: dict) -> None:
        from math import sqrt
        _LOG.warning('%s', data)
        _LOG.warning('%s', locals())
        processed_data = {_: eval(self._metrics_formulas[_], globals(), locals()) for _ in data}
        _LOG.warning('%s', processed_data)
        return processed_data

    def _add_measurements(self, measurements: ET.Element, name, *, depth: int) -> None:
        data = {self._metrics_by_id[int(measurement.attrib['n'])]: float(measurement.attrib['v'])
                for measurement in measurements if measurement.tag == 'M'}
        self._evaluate_measurements_data(data)
        series = pd.Series(data=data, name=name)
        self.loc[name] = series

        if depth >= 6:
            return

        for measurement in measurements:
            if measurement.tag == 'M':
                continue
            if measurement.tag == 'PF':
            #assert measurement.tag == 'PF', measurement.tag
                print(measurement.attrib)
                self._add_measurements(measurement, self._procedures_by_id[int(measurement.attrib['n'])], depth=depth + 1)
            if measurement.tag == 'C':
                print(measurement.attrib)
                self._add_measurements(measurement, '{}.{}'.format(name, measurement.attrib['s']), depth=depth + 1)
            #raise NotImplementedError((measurement.tag, measurement.attrib))

In [309]:
df = HPCtoolkitDB(path=profile_db_path(
    datetime.datetime(2018, 9, 5, 12, 5, 19, 934255), test_name='subset_Sedov_base_a').joinpath('experiment.xml'))

flash4
{'CPUTIME (usec):CfVar (E)': 'sqrt((data[self._metrics_by_id[118]] / '
                             'data[self._metrics_by_id[119]]) - '
                             'pow(data[self._metrics_by_id[117]] / '
                             'data[self._metrics_by_id[119]], 2)) / '
                             '(data[self._metrics_by_id[117]] / '
                             'data[self._metrics_by_id[119]])',
 'CPUTIME (usec):CfVar (I)': 'sqrt((data[self._metrics_by_id[107]] / '
                             'data[self._metrics_by_id[108]]) - '
                             'pow(data[self._metrics_by_id[106]] / '
                             'data[self._metrics_by_id[108]], 2)) / '
                             '(data[self._metrics_by_id[106]] / '
                             'data[self._metrics_by_id[108]])',
 'CPUTIME (usec):CfVar:num-src (E)': 'data[self._metrics_by_id[119]]',
 'CPUTIME (usec):CfVar:num-src (I)': 'data[self._metrics_by_id[108]]',
 'CPUTIME (usec):Max (E)': 'data[self._

NameError: name 'data' is not defined

In [259]:
df[['CPUTIME (usec):Sum (I)', 'CPUTIME (usec):Mean (I)', 'CPUTIME (usec):Min (I)', 'CPUTIME (usec):Max (I)']]

,CPUTIME (usec):Sum (I),CPUTIME (usec):Mean (I),CPUTIME (usec):Min (I),CPUTIME (usec):Max (I)
global,977781000.0,977781000.0,19303000.0,19867400.0
<partial call paths>,640477.0,640477.0,7957.0,47896.0
<partial call paths>.13379,640477.0,640477.0,7957.0,47896.0
<unknown procedure>,640477.0,640477.0,7957.0,47896.0
<program root>,977141000.0,977141000.0,19303000.0,19859400.0
<program root>.13385,977141000.0,977141000.0,19303000.0,19859400.0
main,977141000.0,977141000.0,19303000.0,19859400.0
main.337,347494.0,347494.0,15861.0,107986.0
driver_initflash_,15850900.0,15850900.0,282393.0,451357.0
driver_initflash_.485,807364.0,807364.0,7972.0,31958.0


In [260]:
df

,CPUTIME (usec):Sum (I),CPUTIME (usec):Mean (I),CPUTIME (usec):Mean:num-src (I),CPUTIME (usec):StdDev (I),CPUTIME (usec):StdDev:accum2 (I),CPUTIME (usec):StdDev:num-src (I),CPUTIME (usec):CfVar (I),CPUTIME (usec):CfVar:accum2 (I),CPUTIME (usec):CfVar:num-src (I),CPUTIME (usec):Min (I),...,CPUTIME (usec):Mean (E),CPUTIME (usec):Mean:num-src (E),CPUTIME (usec):StdDev (E),CPUTIME (usec):StdDev:accum2 (E),CPUTIME (usec):StdDev:num-src (E),CPUTIME (usec):CfVar (E),CPUTIME (usec):CfVar:accum2 (E),CPUTIME (usec):CfVar:num-src (E),CPUTIME (usec):Min (E),CPUTIME (usec):Max (E)
global,977781000.0,977781000.0,50.0,977781000.0,1.912200e+16,50.0,977781000.0,1.912200e+16,50.0,19303000.0,...,NaN,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN
<partial call paths>,640477.0,640477.0,50.0,640477.0,1.356710e+10,50.0,640477.0,1.356710e+10,50.0,7957.0,...,NaN,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN
<partial call paths>.13379,640477.0,640477.0,50.0,640477.0,1.356710e+10,50.0,640477.0,1.356710e+10,50.0,7957.0,...,NaN,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN
<unknown procedure>,640477.0,640477.0,50.0,640477.0,1.356710e+10,50.0,640477.0,1.356710e+10,50.0,7957.0,...,640477.0,50.0,640477.0,1.356710e+10,50.0,640477.0,1.356710e+10,50.0,7.957000e+03,47896.0
<program root>,977141000.0,977141000.0,50.0,977141000.0,1.909700e+16,50.0,977141000.0,1.909700e+16,50.0,19303000.0,...,NaN,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN
<program root>.13385,977141000.0,977141000.0,50.0,977141000.0,1.909700e+16,50.0,977141000.0,1.909700e+16,50.0,19303000.0,...,NaN,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN
main,977141000.0,977141000.0,50.0,977141000.0,1.909700e+16,50.0,977141000.0,1.909700e+16,50.0,19303000.0,...,NaN,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN
main.337,347494.0,347494.0,50.0,347494.0,2.096390e+10,50.0,347494.0,2.096390e+10,50.0,15861.0,...,NaN,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN
driver_initflash_,15850900.0,15850900.0,50.0,15850900.0,5.072530e+12,50.0,15850900.0,5.072530e+12,50.0,282393.0,...,NaN,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN
driver_initflash_.485,807364.0,807364.0,50.0,807364.0,1.664980e+10,50.0,807364.0,1.664980e+10,50.0,7972.0,...,NaN,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN
